In [11]:
import logging
from elasticsearch import Elasticsearch

In [25]:
def connect_elasticsearch():
    _es = None
    _es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
    if _es.ping():
        print('Yay Connect')
    else:
        print('Awww it could not connect!')
    return _es
if __name__ == '__main__':
    logging.basicConfig(level=logging.ERROR)

In [31]:
def create_index(es_object, index_name):
    created = False
    # index settings
#     settings = {
#         "settings": {
#             "number_of_shards": 1,
#             "number_of_replicas": 0
#         },
#         "mappings": {
#             "testindex": {
#                 "dynamic": "strict",
#                 "properties": {
#                     "title": {
#                         "type": "text"
#                     },
#                     "description": {
#                         "type": "text"
#                     },
#                     "calories": {
#                         "type": "integer"
#                     }
#                 }
#             }
#         }
#     }
    settings = {
            "settings": {
                "number_of_shards": 1,
                "number_of_replicas": 0
            },
            "mappings": {
                "test1": {
                    "dynamic": "strict",
                    "properties": {
                        "url": {
                            "type": "text"
                        },
                        "text": {
                            "type": "text"
                        }
                    }
                }
            }
        }

    try:
        if not es_object.indices.exists(index_name):
            # Ignore 400 means to ignore "Index Already Exist" error.
            es_object.indices.create(index=index_name, ignore=400, body=settings)
            print('Created Index')
        created = True
    except Exception as ex:
        print(str(ex))
    finally:
        return created

In [37]:
es = connect_elasticsearch()

Yay Connect


In [33]:
create_index(connect_elasticsearch(), 'test1')

Yay Connect


True

In [21]:
def search(es_object, index_name, search):
    res = es_object.search(index=index_name, body=search)
    pprint(res)

In [22]:
def store_record(elastic_object, index_name, record):
    is_stored = True
    try:
        
        outcome = elastic_object.index(index=index_name, doc_type='salads', body=record)
        print(outcome)
    except Exception as ex:
        print('Error in indexing data')
        print(str(ex))
        is_stored = False
    finally:
        return is_stored

In [30]:
store_record(es, 'test1', {'title':'this is title', 'description':'this is desc', 'calories': 456})

{'_index': 'test1', '_type': 'salads', '_id': 'rNio524BaOUwWxp_Kd90', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}


True

In [36]:
store_record(es, 'test2', {'url':'this is title', 'text':'this is desc'})

{'_index': 'test2', '_type': 'salads', '_id': 'rtiz524BaOUwWxp_QN_V', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}


True

https://stackoverflow.com/questions/30859142/how-to-delete-documents-from-elasticsearch'


In [ ]:
 db.delete_by_query(index='reestr',doc_type='some_type', q={'name': 'Jacobian'})

In [ ]:
es.indices.delete(index='test-index', ignore=[400, 404])

In [39]:
es.indices.delete(index='testsites', ignore=[400, 404])

{'acknowledged': True}

### Parse PDF files

In [42]:
from tika import parser

In [44]:
raw = parser.from_file('data/Badge key form.pdf')

AttributeError: 'bytes' object has no attribute 'close'

In [ ]:
print(raw['content'])